In [ ]:
from flask import Flask, render_template, request, session, url_for, redirect, jsonify
import pymysql

#=========
import nltk
from nltk.stem.lancaster import LancasterStemmer

stemmer = LancasterStemmer()
import numpy
import tflearn
import tensorflow
import json
import pickle

from sklearn import preprocessing
import pandas as pd
import numpy as np
import scipy.stats
import scipy.spatial
from sklearn.model_selection import KFold
import random
from sklearn.metrics import mean_squared_error
from math import sqrt
import math
import warnings
import sys
#from sklearn.utils.extmath import np.dot

#=========Database Connection===
connection = pymysql.connect(host="localhost", user="root", password="", database="063autoqa")
cursor = connection.cursor()
#============start=======chatbot============
#chatbot
with open("QASystemCrypto.json", encoding="utf8") as file:
    data = json.load(file)

try:
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
except:
    words = []
    labels = []
    docs_x = []
    docs_y = []

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            # print(wrds)
            docs_y.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))

    labels = sorted(labels)

    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []

        wrds = [stemmer.stem(w.lower()) for w in doc]

        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)

    training = numpy.array(training)
    output = numpy.array(output)

    with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)

tensorflow.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model11 = tflearn.DNN(net)

try:
    model11.load("model.tflearn")
except:

    model11.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
    model11.save("model.tflearn")


#==============end====chatbot=================


app = Flask(__name__)
app.secret_key = 'random string'


@app.route('/index')
@app.route('/')
def index():
    return render_template('index.html')


@app.route('/register', methods=["GET","POST"])
def register():
    if request.method == "POST":
        name = request.form.get("name")
        email = request.form.get("email")
        mobile = request.form.get("mobile")
        gender = request.form.get("gender")
        dob = request.form.get("dob")
        username = request.form.get("username")
        password = request.form.get("password")
        cursor.execute("insert into userdetails(fullname,gender,mobile,email,dob,username,password) values('"+name+"','"+gender+"','"+mobile+"','"+email+"','"+dob+"','"+username+"','"+password+"')")
        connection.commit()
        #return render_template('/index')
        return redirect(url_for('/index'))
    else:
        #return render_template('index.html')
        return redirect(url_for('/index'))


@app.route('/login', methods=["GET","POST"])
def login():
    msg = ''
    if request.method == "POST":
        session.pop('user',None)
        username = request.form.get("username")
        password = request.form.get("password")
        cursor.execute('SELECT * FROM userdetails WHERE username = %s AND password = %s', (username, password))
        account = cursor.fetchone()
        #print(account)
        if account:
            session['user'] = account[1]
            #return render_template('home.html')
            return redirect(url_for('duplicatedetection'))
        else:
            # Account doesnt exist or username/password incorrect
            msg = 'Incorrect username/password!'
    #return render_template('index.html', msg=msg)
    return redirect(url_for('index'))


#logout code
@app.route('/logout')
def logout():
    session.pop('user')
    return redirect(url_for('index'))


@app.route('/home')
def home():
    if 'user' in session:
        return render_template('home.html', user=session['user'])
    return redirect(url_for('index'))

@app.route('/aboutus')
def aboutus():
    return render_template('aboutus.html')


@app.route('/contactus')
def contactus():
    return render_template('contactus.html')
#bot start======

@app.route('/duplicatedetection')
def duplicatedetection():
    if 'user' in session:
        return render_template('duplicatedetection.html', user=session['user'])
    return redirect(url_for('index'))


@app.route("/get")
def get_bot_response():
    userText = request.args.get('msg')
    results = model11.predict([bag_of_words(userText, words)])
    print(results)
    results_index = numpy.argmax(results)
    print(results_index)
    print(results[0][results_index])
    oppred=results[0][results_index]
    outdatagot = "No duplicate Question found for this query"
    if oppred>0.7:
        tag = labels[results_index]
        #outdatagot = ""
        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['patterns']
                responseop=""
                for ik in responses:
                    responseop=responseop+ik+","
                outdatagot = "input matches with question \n "+responseop+" with "+str(oppred)#random.choice(responses)
                print(outdatagot)
        return str(outdatagot)
    else:
        outdatagot=outdatagot+" "+str(oppred)
        return str(outdatagot)


def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]
    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
    return numpy.array(bag)
model11.load("model.tflearn")
#bot end======-------------


if __name__ == '__main__':
    #app.run(debug="True")
    app.run('0.0.0.0')


C:\Users\Ningesh\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Ningesh\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Ningesh\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Ningesh\AppData\Roaming\Python\Python

hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)
curses is not supported on this machine (please install/reinstall curses for an optimal experience)







Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Instructions for updating:
keep_dims is deprecated, use keepdims instead

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:\Users\Ningesh\049 AutoQA\model.tflearn
INFO:tensorflow:Restoring parameters from C:\Users\Ningesh\049 AutoQA\model.tflearn
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Jul/2020 13:37:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2020 13:37:13] "POST /login HTTP/1.1" 302 -
127.0.0.1 - - [10/Jul/2020 13:37:13] "GET /duplicatedetection HTTP/1.1" 200 -
127.0.0.1 - - [10/Jul/2020 13:37:26] "GET /get?msg=what+is+machine+learning HTTP/1.1" 200 -


[[3.8802510e-33 1.0286125e-31 0.0000000e+00 1.2716047e-34 2.7784532e-22
  6.4642446e-20 0.0000000e+00 1.2292070e-37 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 1.3073921e-36
  1.6498924e-32 0.0000000e+00 0.0000000e+00 6.4967318e-19 3.5595696e-15
  3.9218307e-17 0.0000000e+00 9.3144378e-14 0.0000000e+00 4.8956329e-20
  0.0000000e+00 0.0000000e+00 3.5553047e-32 2.7539063e-24 0.0000000e+00
  0.0000000e+00 5.8856437e-37 4.4621715e-19 5.7120771e-36 2.4814686e-05
  5.0090193e-26 1.3501795e-22 1.9131768e-04 0.0000000e+00 0.0000000e+00
  0.0000000e+00 0.0000000e+00 8.5169262e-18 0.0000000e+00 6.1485093e-37
  0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 1.1944880e-36
  0.0000000e+00 5.0879610e-25 2.0018779e-15 9.8209733e-01 2.2300412e-35
  0.0000000e+00 5.0219925e-33 4.5246010e-25 4.7885258e-19 1.7120341e-02
  1.4462055e-16 1.7133576e-05 1.0365868e-20 6.5800270e-07 1.1324510e-06
  1.3208696e-17 2.1947089e-32 0.0000000e+00 1.5392002e-33 5.3750

127.0.0.1 - - [10/Jul/2020 13:37:37] "GET /get?msg=python HTTP/1.1" 200 -


...

[[0.00000000e+00 1.71939161e-25 4.61210778e-23 1.90729224e-25
  6.15954863e-22 7.64117174e-26 0.00000000e+00 2.40371284e-34
  2.72060747e-26 2.72623903e-18 5.60944723e-14 1.72867480e-21
  4.46449353e-18 1.28157407e-27 1.20431754e-20 2.02545317e-31
  5.94927847e-31 5.87898062e-31 3.47707851e-06 8.51390948e-19
  2.38216476e-16 2.38381932e-31 5.41356879e-33 0.00000000e+00
  6.77785636e-07 1.11935568e-23 2.64702989e-35 0.00000000e+00
  2.26306877e-37 5.07160428e-20 2.34293535e-23 2.38353010e-07
  1.71349217e-25 0.00000000e+00 1.49898536e-21 0.00000000e+00
  5.91531402e-30 1.04540199e-08 2.36027487e-31 1.10456174e-36
  2.18644989e-29 9.13495773e-35 5.14665220e-11 0.00000000e+00
  2.99263961e-15 2.48657339e-24 3.44570548e-36 8.44171742e-21
  3.59940682e-28 6.26491761e-26 4.30377427e-26 2.21918711e-07
  3.11862639e-30 1.46953175e-22 8.12885625e-36 1.84415203e-18
  3.32369820e-13 1.06590925e-11 7.00200076e-08 4.49863045e-08
  5.41726969e-17 4.76134682e-22 2.55162953e-37 1.06437421e-32
  8.9387

127.0.0.1 - - [10/Jul/2020 13:39:31] "GET /get?msg=What+Is+Pragmatic+Analysis+In+Nlp HTTP/1.1" 200 -


[[7.89903434e-25 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 5.20439889e-25 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 1.84214562e-23 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 5.26825861e-10
  1.70044944e-35 0.00000000e+00 0.00000000e+00 0.00000000e+00
  1.40360343e-37 1.52195234e-08 3.21031715e-31 3.71917892e-08
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 2.77195894e-37
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 1.87079937e-37 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 1.95740087e-26 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 4.29029026e-33 7.34887989e-24
  0.0000

127.0.0.1 - - [10/Jul/2020 13:39:48] "GET /get?msg=what+is+Artificial+Intelligence HTTP/1.1" 200 -


[[8.00933605e-38 0.00000000e+00 0.00000000e+00 1.18723857e-16
  0.00000000e+00 0.00000000e+00 0.00000000e+00 9.17217459e-31
  0.00000000e+00 1.64086757e-35 2.93127401e-34 1.91236784e-22
  1.21556754e-04 0.00000000e+00 0.00000000e+00 0.00000000e+00
  2.18714573e-13 2.82280689e-06 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  2.85243519e-35 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 9.99763310e-01 5.68399818e-16
  5.87775482e-38 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 3.02684060e-18 1.07972208e-19
  1.42860568e-26 1.71834534e-24 0.00000000e+00 0.00000000e+00
  6.25046198e-25 2.11722848e-35 0.00000000e+00 2.63596487e-08
  1.11809175e-04 0.00000000e+00 1.40703893e-08 0.00000000e+00
  0.00000000e+00 0.00000000e+00 5.02288292e-26 2.10593313e-07
  2.24848978e-26 0.00000000e+00 8.13509859e-29 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.0000